In [106]:
import numpy as np
import pickle
import matplotlib.pyplot as plt

def voltage_time_figure_plot(X1, Y1, color_code, name):
    # Create figure
    figure1 = plt.figure(name)
    
    # Create axes
    axes1 = figure1.add_subplot(1, 1, 1)
    
    # Create plot
    axes1.plot(X1, Y1, linewidth=2, color=color_code)
    
    # Set ylabel
    axes1.set_ylabel('Voltage (mV)', fontsize=16.5)
    
    # Set xlabel
    axes1.set_xlabel('Time (s)', fontsize=16.5)
    
    # Set title
    axes1.set_title(name, fontsize=16.5)
    
    # Set x-axis limit
    # axes1.set_xlim([0, 0.000001])
    
    # Uncomment the following line to set y-axis limit
    # axes1.set_ylim([-60, 60])
    
    # Uncomment the following line to set z-axis limit
    # axes1.set_zlim([-1, 1])
    
    axes1.grid(True)
    axes1.set_axisbelow(True)
    axes1.tick_params(labelsize=15)
    
    # Show the plot
    plt.show()

def set_mean(waveform):
    return np.mean(waveform[1], axis=0)

def set_offset(waveform, adjust):
    return np.array(waveform[1]) - waveform[1][0] * adjust

def set_gain(waveform, adjust):
    gain = np.abs(np.max(waveform[1]) * adjust)
    return np.array(waveform[1]) / gain

def set_gain_tail(waveform, time_point):
    time_point_index = np.where(np.array(waveform[0]) > time_point)[0][0]
    gain = np.abs(waveform[1][time_point_index])
    print("gain = {}".format(gain))
    print("waveform[1][time_point_index] = {}".format(np.array(waveform[1])[time_point_index]))
    print("np.array(waveform[1]) / gain = {}".format(np.array(waveform[1])[time_point_index] / gain))
    return np.array(waveform[1]) / gain

def moving_agerage(waveform,window_size ):
    i = 0
    # Initialize an empty list to store moving averages
    moving_averages = []
    arr = np.array(waveform[1])
    # Loop through the array t o
    while i < len(arr) - window_size + 1:
    
        # Calculate the average of current window
        window_average = np.sum(arr[
        i:i+window_size]) / window_size
        
        # Store the average of current
        # window in moving average list
        moving_averages.append(window_average)
        
        # Shift window to right by one position
        i += 1
    waveform[0] = np.array(waveform[0])[0 : len(arr) - window_size+1]
    waveform[1] = moving_averages
    
    return waveform

def append_waveform_index(waveform, index, name, file_name, time_point):
    waveform_dummy = pickle.load(open(file_name, "rb"))# set_mean(pickle.load(open(file_name, "rb")))# moving_agerage(pickle.load(open(file_name, "rb")), window_size)
    # print(np.shape(waveform_dummy))
    # print(waveform_dummy)
    waveform_dummy[1] = set_mean(waveform_dummy)
    waveform_dummy[0] = waveform_dummy[0][0]
    waveform_dummy = moving_agerage(waveform_dummy, window_size)
    waveform_dummy[1] = set_offset(waveform_dummy,1)
    # waveform_dummy[1] = set_gain(waveform_dummy, 1)
    waveform_dummy[1] = set_gain_tail(waveform_dummy, time_point)
    index.append(name)
    waveform.append(waveform_dummy)
    
    return waveform, index

def plot_list(ax, index_want_plot, index_set, waveform, X, header_label):
    for line_code in index_want_plot:
        i = np.where( np.array(index_set) == line_code)[0][0]
        ax.plot(X, waveform[i][1], label= header_label + index_set[i])
    return 0

In [145]:
window_size = 500
time_point = 109 * 10**(-6)

character_set = ["A", "B", "C", "D", "E", "F"]
atten_db_set = [None ,"0", "10", "20", "0", None]

waveform = [] # Waveform with cold plate only
waveform_AttenPlate = [] # Waveform cold plate plus attenuator (BB, CC, DD, EE) = (0dB, 10dB, 20dB, 0dB)
waveform_AttenOnly = [] # Wavefor attenuator only (0dB, 10dB, 20dB)
waveform_CopperLine = [] # Waveform copper line only with three different length
waveform_AttenCopperLine = []

# The corresponding string index 
index = []
index_AttenPlate = []
index_AttenOnly = []
index_CopperLine = []
index_AttenCopperLine = []

# Load waveform of all components
length = len(character_set)
for index_chara1 in range(length):
    chara1= character_set[index_chara1]
    file_name = "E:/OP/Experiments/0616_2023_pred/DATA/test/copper_line/Line"+ chara1 + chara1 +"_10rep_80G_80kpts_70us_TLineCase2.pkl"
    waveform, index = append_waveform_index(waveform, index, chara1 + chara1, file_name, time_point)

    if chara1 not in ["A", "F"]:
        atten_name = atten_db_set[index_chara1]
        file_name = "E:/OP/Experiments/0616_2023_pred/DATA/test/copper_line/Line"+ chara1 + chara1 +"_" + atten_name +"dbAtten_10rep_80G_80kpts_70us_TLineCase2.pkl"
        waveform_AttenPlate, index_AttenPlate =  append_waveform_index(waveform_AttenPlate, index_AttenPlate, chara1 + chara1, file_name, time_point)
        if chara1 not in ["E"]:
            file_name = "E:/OP/Experiments/0616_2023_pred/DATA/test/copper_line/" + atten_name + "dbAtten_10rep_80G_80kpts_70us_TLineCase2.pkl"
            waveform_AttenOnly, index_AttenOnly =  append_waveform_index(waveform_AttenOnly, index_AttenOnly, atten_name, file_name, time_point)
    for index_chara2 in range(index_chara1, length):
        chara2= character_set[index_chara2]
        if chara1 + chara2 in ['BC', 'CD', 'DE']:
            file_name = "E:/OP/Experiments/0616_2023_pred/DATA/test/copper_line/CopperLine"+ chara1 + chara2 +"_10rep_80G_80kpts_70us_TLineCase2.pkl"
            waveform_CopperLine, index_CopperLine = append_waveform_index(waveform_CopperLine, index_CopperLine, chara1 + chara2, file_name, time_point)  
            file_name = "E:/OP/Experiments/0616_2023_pred/DATA/test/copper_line/"+chara1+chara1+"Atten_CopperLine"+chara1+chara2+"_"+chara2+chara2+"Atten_10rep_80G_80kpts_70us_TLineCase2.pkl"
            waveform_AttenCopperLine, index_AttenCopperLine = append_waveform_index(waveform_AttenCopperLine, index_AttenCopperLine, chara1+chara1+chara1 + chara2+chara2+chara2, file_name, time_point) 
# Read AWG waveform
waveform_AWG =  pickle.load(open("E:/OP/Experiments/0616_2023_pred/DATA/test/copper_line/BareAWG_10rep_80G_80kpts_70us_TLineCase2.pkl", "rb"))# moving_agerage(pickle.load(open("E:/OP/Experiments/0616_2023_pred/DATA/retest_500smooth_BareAWG_new_averaged.pkl", "rb")), window_size)
waveform_AWG[1] = set_mean(np.copy(waveform_AWG))
waveform_AWG[0] = waveform_AWG[0][0]
waveform_AWG = moving_agerage(waveform_AWG, window_size)
waveform_AWG[1] = set_offset(np.copy(waveform_AWG),1)
# waveform_AWG[1] = set_gain(np.copy(waveform_AWG),1)
waveform_AWG[1] = set_gain_tail(np.copy(waveform_AWG), time_point)

# waveform_AWG2 =  pickle.load(open("E:/OP/Experiments/0616_2023_pred/DATA/test/copper_line/test1_BareAWG_10rep_80G_80kpts_70us_TLineCase2.pkl", "rb"))# moving_agerage(pickle.load(open("E:/OP/Experiments/0616_2023_pred/DATA/retest_500smooth_BareAWG_new_averaged.pkl", "rb")), window_size)
# waveform_AWG2[1] = set_mean(np.copy(waveform_AWG2))
# waveform_AWG2[0] = waveform_AWG2[0][0]
# waveform_AWG2 = moving_agerage(waveform_AWG2, window_size)
# waveform_AWG2[1] = set_offset(np.copy(waveform_AWG2),1)
# waveform_AWG2[1] = set_gain_tail(np.copy(waveform_AWG2), time_point)

# waveform_AWG = waveform_AWG

index_L0 = ['AA', 'BB', 'CC', 'DD', 'EE', 'FF']
index_L1 = ['AB', 'BC', 'CD', 'DE', 'EF']
index_L2 = ['AC', 'BD', 'CE', 'DF']
index_L3 = ['AD', 'BE', 'CF']
index_L4 = ['AE', 'BF']
index_L5 = ['AF']

X = np.array(waveform[0][0]) * 10**6

gain = 39405.662
waveform[1][time_point_index] = 39405.662
np.array(waveform[1]) / gain = 1.0
gain = 39322.207839999995
waveform[1][time_point_index] = 39322.207839999995
np.array(waveform[1]) / gain = 1.0
gain = 13085.86736
waveform[1][time_point_index] = 13085.86736
np.array(waveform[1]) / gain = 1.0
gain = 9837.6332
waveform[1][time_point_index] = 9837.6332
np.array(waveform[1]) / gain = 1.0
gain = 38552.13604
waveform[1][time_point_index] = 38552.13604
np.array(waveform[1]) / gain = 1.0
gain = 6278.974200000001
waveform[1][time_point_index] = 6278.974200000001
np.array(waveform[1]) / gain = 1.0
gain = 13144.23404
waveform[1][time_point_index] = 13144.23404
np.array(waveform[1]) / gain = 1.0
gain = 12808.440159999998
waveform[1][time_point_index] = 12808.440159999998
np.array(waveform[1]) / gain = 1.0
gain = 12694.66424
waveform[1][time_point_index] = 12694.66424
np.array(waveform[1]) / gain = 1.0
gain = 38341.091400000005
waveform[1][time_point_index] = 38341.091400000005
np.array(

In [148]:
print("Index: {}, Cold plate only".format(index))
print("Index_AttenPlate: {}, Cold plate + attenuator (DD: 20dB; BB and EE: 0dB; CC: 10dB)".format(index_AttenPlate))
print("Index_AttenOnly: {}, attenuator only (20dB, 0dB, and 10dB)".format(index_AttenOnly))
print("Index_CopperLine: {}, Copper line only".format(index_CopperLine))
print("waveform_AttenCopperLine: {}, Copper line only".format(index_AttenCopperLine))

Index: ['AA', 'BB', 'CC', 'DD', 'EE', 'FF'], Cold plate only
Index_AttenPlate: ['BB', 'CC', 'DD', 'EE'], Cold plate + attenuator (DD: 20dB; BB and EE: 0dB; CC: 10dB)
Index_AttenOnly: ['0', '10', '20'], attenuator only (20dB, 0dB, and 10dB)
Index_CopperLine: ['BC', 'CD', 'DE'], Copper line only
waveform_AttenCopperLine: ['BBBCCC', 'CCCDDD', 'DDDEEE'], Copper line only


# Plotting 

In [154]:
index_want_plot1 = [ 'DD', 'EE'] #['BB'] # ['AA', 'BB', 'CC', 'DD', 'EE', 'FF'] # # for the waveform case # plate only
index_want_plot2 = [None] # ['BB'] #['BB', 'CC', 'DD', 'EE']  # for the waveform_AttenPlate case
index_want_plot3 = ['20', '0'] # ['0'] # ['0', '10', '20'] # for the waveform_AttenOnly
index_want_plot4 = ['DE'] # ['BC'] # ['BC', 'CD', 'DE'] # for the waveform_CopperLine case
index_want_plot5 = ['DDDEEE'] # ['BBBCCC', 'CCCDDD', 'DDDEEE'] # for the waveform_AttenCopperLine case

index_want_plot_total = [index_want_plot1, index_want_plot2, index_want_plot3, index_want_plot4, index_want_plot5]
index_total = [index, index_AttenPlate, index_AttenOnly, index_CopperLine, index_AttenCopperLine]
waveform_total = [waveform, waveform_AttenPlate, waveform_AttenOnly, waveform_CopperLine, waveform_AttenCopperLine]
header_label_total = [" Cold plate only ", "Cold plate + attenuator ", "Attenuator only (dB) ", "Copper line only ", "Atten + plate + line + atten + plate "]

AWG_switch = 1 # 0 off, 1 on

%matplotlib qt

fig = plt.figure()
ax = fig.add_subplot(111)

for jdex in range(len(index_want_plot_total)):
    if None not in index_want_plot_total[jdex]:
        plot_list(ax, index_want_plot_total[jdex],  index_total[jdex], waveform_total[jdex], X, header_label_total[jdex])
if AWG_switch == 1:
    ax.plot(X, waveform_AWG[1], label='AWG_Bare')
    # ax.plot(X, waveform_AWG2[1], label='AWG_Bare test2')
ax.grid()

# Labels and title
ax.set_xlabel('Time (us)')
ax.set_ylabel('Amplitude')
ax.set_title('Distorted waveform on copper lines, cable = 2.5m')

# Legend
ax.legend()

# Display the plot
plt.show()

In [113]:
np.median(np.array(X)) - X[0]

39.75

In [126]:
39.533206 - 39.499206

0.03399999999999892